# Test jupyter
This jupyter notebook is to generate a submission.

In [3]:
import Levenshtein
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import shap
import warnings
import xgboost



from pandarallel import pandarallel
from scipy.stats import spearmanr

In [4]:
# Configuration
pandarallel.initialize(use_memory_fs=False)
warnings.filterwarnings('ignore')

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [5]:
raw_data_path = '../data/raw/'

In [6]:
# You can get this data from: 
#https://www.kaggle.com/datasets/alejopaullier/aminoacids-physical-and-chemical-properties

aminoacids = pd.read_csv(f'{raw_data_path}aminoacids.csv')
aminoacids.rename(
    columns={
        col: col.lower().strip().replace(' ', '_') 
        for col in aminoacids.columns
    }, inplace=True
)

protein_lists_pbd = pd.read_csv(
    f'{raw_data_path}similar_protein_sequences.csv', 
    index_col="seq_id"
)

In [37]:
original_sequence = 'VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVDCWAILCKGAPNVLQRVNEKTKNSNRDRSGANKGPFKDPQKWGIKALPPKNPSWSAQDFKSPEEYAFASSLQGGTNAILAPVNLASQNSQGGVLNGFYSANKVAQFDPSKPQQTKGTWFQITKFTGAAGPYCKALGSNDKSVCDKNKNIAGDWGFDPAKWAYQYDEKNNKFNYVGK'
df_test = pd.read_csv(f"{raw_data_path}test.csv", index_col="seq_id")
df_test

,protein_sequence,pH,data_source
seq_id,,,
31390,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
31391,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
31392,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,Novozymes
31393,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
31394,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
...,...,...,...
33798,VPVNPEPDATSVENVILKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
33799,VPVNPEPDATSVENVLLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes
33800,VPVNPEPDATSVENVNLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,Novozymes


In [38]:
def change_positions(protein1, protein2):
    try:
        edit_operations = Levenshtein.editops(protein1, protein2)
        result_positions = [element[1] for element in edit_operations]
        return int(result_positions[0])
    except:
        return None

def get_wildtype(pos):
    try: 
        pos=int(pos)
        return original_sequence[pos]
    except:
        return None
def get_mutation(df):
    try:
        mutation = df.protein_sequence[int(df.change_positions)]
        return mutation
    except:
        return None

In [39]:
df_test['change_positions'] =df_test.protein_sequence.apply(lambda x: change_positions(original_sequence, x))
df_test['seq1'] = original_sequence
df_test['pH1'] = 8
df_test['len1'] = len(original_sequence)
df_test['seq2'] = df_test.protein_sequence
df_test['pH2'] = df_test.pH
df_test['len2'] = df_test.protein_sequence.str.len()
df_test['mutation_type'] = 1
df_test['wildtype'] = df_test.change_positions.apply(get_wildtype)
df_test['mutation'] = df_test.apply(get_mutation, axis=1)
df_test['edit_operations'] =df_test['change_positions']

In [50]:
drop_cols = ['protein_sequence', 'pH', 'data_source', 'change_positions', 'len2']
temporal_data = df_test[df_test.mutation.notna()].drop(columns=drop_cols)

In [51]:
temporal_data['mutation_position'] =  temporal_data['edit_operations']
temporal_data['relative_mutation_position'] = temporal_data['mutation_position']/temporal_data['len1']
temporal_data['d_pH'] = temporal_data['pH2'] - temporal_data['pH1']

In [52]:
def aminoacid_data(row):
    letter_wildtype = row.wildtype[0]
    letter_mutation = row.mutation[0]
    wild_amino = aminoacids[aminoacids.letter == letter_wildtype]
    muta_amino = aminoacids[aminoacids.letter == letter_mutation]
    cols = [
        'molecular_weight',
        'residue_weight',
        'pka1',
        'pkb2',
        'pkx3',
        'pl4',
        'h',
        'vsc',
        'p1',
        'p2',
        'sasa',
        'ncisc',
        'carbon',
        'hydrogen',
        'nitrogen',
        'oxygen',
        'sulfur'
    ]
    difference = (
        muta_amino[cols].reset_index(drop=True).T - 
        wild_amino[cols].reset_index(drop=True).T
    )
    result = dict(difference[0])
    result = {f'{key}__difference': result[key] for key in result.keys()}
    return result

In [60]:
temporal_data

,seq1,pH1,len1,seq2,pH2,mutation_type,wildtype,mutation,edit_operations,mutation_position,relative_mutation_position,d_pH
seq_id,,,,,,,,,,,,
31390,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,221,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,1,L,E,16.0,16.0,0.072398,0
31391,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,221,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,1,L,K,16.0,16.0,0.072398,0
31392,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,221,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,1,L,K,16.0,16.0,0.072398,0
31393,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,221,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,1,K,C,17.0,17.0,0.076923,0
31394,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,221,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,1,K,F,17.0,17.0,0.076923,0
...,...,...,...,...,...,...,...,...,...,...,...,...
33798,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,221,VPVNPEPDATSVENVILKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,1,A,I,15.0,15.0,0.067873,0
33799,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,221,VPVNPEPDATSVENVLLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,1,A,L,15.0,15.0,0.067873,0
33800,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,221,VPVNPEPDATSVENVNLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,1,A,N,15.0,15.0,0.067873,0


In [62]:
temporal_data_aminoacid_info = pd.json_normalize(
    temporal_data.parallel_apply(
        aminoacid_data, axis=1
    )
)
temporal_data_aminoacid_info.set_index(temporal_data.index, inplace=True)

In [63]:
drop_cols = [
    'mutation', 
    'wildtype', 
    'edit_operations',
    'mutation_position',
]
final_train_data = pd.concat([
    temporal_data.drop(drop_cols, axis=1),
    temporal_data_aminoacid_info], axis=1)
final_train_data

,seq1,pH1,len1,seq2,pH2,mutation_type,relative_mutation_position,d_pH,molecular_weight__difference,residue_weight__difference,...,vsc__difference,p1__difference,p2__difference,sasa__difference,ncisc__difference,carbon__difference,hydrogen__difference,nitrogen__difference,oxygen__difference,sulfur__difference
seq_id,,,,,,,,,,,,,,,,,,,,,
31390,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,221,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,1,0.072398,0,15.95,15.96,...,-31.5,7.4,-0.035,-0.069,-0.044870,-1.0,-4.0,0.0,2.0,0.0
31391,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,221,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,1,0.072398,0,15.01,15.02,...,6.5,6.4,0.033,0.327,-0.033964,0.0,1.0,1.0,0.0,0.0
31392,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,221,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,1,0.072398,0,15.01,15.02,...,6.5,6.4,0.033,0.327,-0.033964,0.0,1.0,1.0,0.0,0.0
31393,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,221,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,1,0.076923,0,-25.03,-25.03,...,-55.4,-5.8,-0.091,-0.797,-0.054318,-3.0,-7.0,-1.0,0.0,1.0
31394,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,221,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,1,0.076923,0,19.00,19.00,...,15.5,-6.1,0.071,-0.030,0.019844,3.0,-3.0,-1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33798,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,221,VPVNPEPDATSVENVILKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,1,0.067873,0,42.08,42.08,...,66.0,-2.9,0.140,0.629,0.014444,3.0,6.0,0.0,0.0,0.0
33799,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,221,VPVNPEPDATSVENVLLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,1,0.067873,0,42.08,42.08,...,66.0,-3.2,0.140,0.750,0.044485,3.0,6.0,0.0,0.0,0.0
33800,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,221,VPVNPEPDATSVENVNLKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,1,0.067873,0,43.02,43.03,...,31.2,3.5,0.088,0.474,-0.001795,1.0,1.0,1.0,1.0,0.0


In [65]:
def generate_graph(protein_sequence):
    from_to_dict = {
        'from':list(protein_sequence[:-1]),
        'to':list(protein_sequence[1:])
    }
    protein_sequence_directions = pd.DataFrame(from_to_dict)
    list_of_paths = protein_sequence_directions.groupby('from').to.agg(list).values.tolist()
    G=nx.Graph()
    for path in list_of_paths:
        nx.add_path(G, path)
    return G
    
def flatten_result_dict(result_dictionary):
    index = []
    values = []
    for key in result_dictionary.keys():
        for letter in result_dictionary[key].keys():
            index.append(f'{key}_{letter}')
            values.append(result_dictionary[key][letter])
    result = pd.Series(values, index=index)
    return result 

def get_graph_results(graph):
    result_dictionary = {
        'degree_dict':dict(graph.degree()),
        'pagerank_dict': nx.pagerank(graph),
        #'centrality_eigenvector': nx.eigenvector_centrality(graph),
        'centrality_degree': nx.degree_centrality(graph),
        'centrality_closeness': nx.closeness_centrality(graph),
    }
    result_dictionary_df = flatten_result_dict(result_dictionary=result_dictionary)
    return result_dictionary_df

In [66]:
graph_seq1 = final_train_data['seq1'].apply(generate_graph).apply(get_graph_results)
graph_seq2 = final_train_data['seq2'].apply(generate_graph).apply(get_graph_results)

In [67]:
for col in graph_seq1.columns:
    final_train_data[f'{col}_graph_diff'] = (
        graph_seq2[col]-graph_seq1[col]
    )
final_train_data.head()

,seq1,pH1,len1,seq2,pH2,mutation_type,relative_mutation_position,d_pH,molecular_weight__difference,residue_weight__difference,...,centrality_closeness_S_graph_diff,centrality_closeness_A_graph_diff,centrality_closeness_G_graph_diff,centrality_closeness_K_graph_diff,centrality_closeness_Y_graph_diff,centrality_closeness_W_graph_diff,centrality_closeness_V_graph_diff,centrality_closeness_C_graph_diff,centrality_closeness_R_graph_diff,centrality_closeness_E_graph_diff
seq_id,,,,,,,,,,,,,,,,,,,,,
31390,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,221,VPVNPEPDATSVENVAEKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,1,0.072398,0,15.95,15.96,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.026154
31391,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,221,VPVNPEPDATSVENVAKKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,1,0.072398,0,15.01,15.02,...,0.0,0.0,0.0,0.044737,0.0,0.000000,0.0,0.000000,0.01828,0.000000
31392,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,221,VPVNPEPDATSVENVAKTGSGDSQSDPIKADLEVKGQSALPFDVDC...,8,1,0.072398,0,15.01,15.02,...,0.0,0.0,0.0,0.044737,0.0,0.000000,0.0,0.000000,0.01828,0.000000
31393,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,221,VPVNPEPDATSVENVALCTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,1,0.076923,0,-25.03,-25.03,...,0.0,0.0,0.0,0.000000,0.0,0.022487,0.0,0.024217,0.00000,0.026154
31394,VPVNPEPDATSVENVALKTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,221,VPVNPEPDATSVENVALFTGSGDSQSDPIKADLEVKGQSALPFDVD...,8,1,0.076923,0,19.00,19.00,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.026154


In [68]:
def aminoacid_window_letter_sum_func(df):
    position = df.relative_mutation_position * len(df.seq1)
    left_position = position - 2
    right_position = position + 2
    aminoacid_window = df.seq1[int(left_position):int(right_position + 1)]
    list_test = []
    for i, letter in enumerate(aminoacid_window):
        if i==2:
            continue
        list_test.append(aminoacids[aminoacids.letter==letter])
    if not list_test:
        return None
    concatenated_letters = pd.concat(list_test)
    total_vecinity = concatenated_letters.select_dtypes('number').sum()
    rival_vecinity = concatenated_letters.select_dtypes('number').head(2).sum()-concatenated_letters.select_dtypes('number').tail(2).sum()
    total_vecinity.rename(index={ix:f'total_sum_neighbour_{ix}' for ix in total_vecinity.index}, inplace=True)
    rival_vecinity.rename(index={ix:f'rival_dif_neighbour_{ix}' for ix in rival_vecinity.index}, inplace=True)
    vecinity_results = total_vecinity.append(rival_vecinity)
    return vecinity_results

In [69]:
vecinity_results = final_train_data.apply(
    aminoacid_window_letter_sum_func, axis=1
)
vecinity_results.set_index(temporal_data.index, inplace=True)

In [70]:
final_train_data = pd.concat([final_train_data, vecinity_results], axis=1)
for i in range(3):
    final_train_data[f'random_{i}'] = np.random.randint(0, 1000, len(final_train_data))

In [72]:
model = xgboost.XGBRegressor()
model.load_model("../models/model1.ubj")

In [84]:
non_train_data = ['seq1', 'seq2']

X_test = final_train_data.drop(columns=non_train_data)

test_data = pd.read_csv('../data/processed/train_data_head.csv', index_col=0)

missing_cols = list(set(test_data.columns) - set(X_test.columns))
for col in missing_cols:
    X_test[col] = np.nan
    
X_test = X_test[test_data.columns]

In [99]:
results = model.predict(X_test)
results_df = pd.DataFrame(pd.Series(results)).set_index(X_test.index)
small_dataframe = pd.DataFrame(pd.Series([0], index=[32559]))
final_results = results_df.append(small_dataframe)


In [104]:
final_results.reset_index().rename(columns={0:'tm', 'index':'seq_id'}).set_index('seq_id').to_csv('../reports/results/submission_0.csv')